In [1]:
import pandas as pd
import numpy as np
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from sklearn.linear_model import LinearRegression
import xgboost as xg
from sklearn.model_selection import GridSearchCV
import time
from sklearn.svm import SVR
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

2023-12-01 17:40:37.484336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
region = 610
train_df = pd.read_csv(f'csv/{region}.csv', index_col=0)
test_df = pd.read_csv(f'csv2/{region}.csv', index_col=0)

In [5]:
def rmse(y_true,y_pred):
    return np.sqrt(np.mean(np.square(y_pred-y_true)))

def mape(y_true, y_pred):
    mask = y_true != 0  # Create a mask for non-zero true values
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.divide(numerator, denominator))

def getTT(region):
    return round((((region-100)//50) * -5.2222 ) + 147.11),round((((region%50)-2) * 5.2083)+20.458 )

def mean_absolute_percentage_error_revise(y_true, y_pred):
    # RMSE
    rmse = K.sqrt(K.mean(K.square(y_pred - y_true)))

    # SMAPE
    epsilon = 1e-10  # Small constant to avoid division by zero
    smape_numerator = tf.abs(y_true - y_pred)
    smape_denominator = tf.maximum(tf.abs(y_true) + tf.abs(y_pred), epsilon)
    smape = K.mean(smape_numerator / smape_denominator)

    alpha = 0.4 # hyperparameter, adjust alpha as needed
    combined_loss = alpha * rmse + (1 - alpha) * smape

    return combined_loss
def custom_metric(y_true, y_pred):
    return mean_absolute_percentage_error_revise(y_true, y_pred)

In [6]:
train = train_df
test = test_df

In [7]:
# polynomial + scale CSV

# create polynomial features
poly = PolynomialFeatures(2,include_bias=False)
train_poly = pd.DataFrame(poly.fit_transform(train),columns=poly.get_feature_names_out(train.columns))
test_poly = pd.DataFrame(poly.fit_transform(test),columns=poly.get_feature_names_out(test.columns))

# initialise scalers
minmaxscaler = MinMaxScaler(feature_range=(0.1,1))
minmaxscaler.fit(train_poly)
minmaxscalerY = MinMaxScaler(feature_range=(0.1,1))
minmaxscalerY.fit((train[[str(region)]]))

# scale train and test using scalers
scaled_train = pd.DataFrame(minmaxscaler.transform(train_poly),columns=train_poly.columns)
scaled_test = pd.DataFrame(minmaxscaler.transform(test_poly),columns=test_poly.columns)

In [8]:
# Create time series for CSV
## For testing
data = scaled_test.values
scaled_data = data

X2, y2 = [], []
sequence_length = 16  # X = 16 previous time steps

for i in range(len(scaled_data) - sequence_length):
    X2.append(scaled_data[i:i+sequence_length, :])  # Use all columns
    y2.append(scaled_data[i+sequence_length, 0])  # Assuming prediction is based on the first column

X2, y2 = np.array(X2), np.array(y2)

## For training
data = scaled_train.values
scaled_data = data

X, y = [], []
sequence_length = 16  # X = 16 previous time steps

for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length, :])  # Use all columns
    y.append(scaled_data[i+sequence_length, 0])  # Assuming prediction is based on the first column

X, y = np.array(X), np.array(y)

In [9]:
train_size = int(len(X) * 0.90)  # 90% training, 10% val
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

In [10]:
X_train.shape

(2664, 16, 324)

### MLP

In [11]:
X = X_train
Y = y_train
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(16, 324), name='input_layer'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid', name='output_layer')
])
model.compile(optimizer='adam', loss=mean_absolute_percentage_error_revise, metrics=[custom_metric])

history = model.fit(X, Y, epochs=25, verbose=1)

Epoch 1/25
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-12-01 17:41:46.341357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [12]:
predictions = model.predict(X2)
print(rmse(y2,predictions))
print(mape(y2,predictions))
print(smape(y2,predictions))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 0s 2ms/step
0.07682666149066743
0.1756792419036884
0.1861012093766768


### XG Boost

In [13]:
X_flattened = X_train.reshape((X_train.shape[0], -1))

In [19]:
xgb_r = xg.XGBRegressor(objective ='reg:logistic', 
                  n_estimators = 5, max_depth=3) 
xgb_r.fit(X_flattened, y_train) 

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=5, n_jobs=0,
             num_parallel_tree=1, objective='reg:logistic', predictor='auto',
             random_state=0, reg_alpha=0, ...)

In [20]:
X2.shape

(2960, 16, 324)

In [21]:
predictions = xgb_r.predict(X2.reshape((X2.shape[0], -1)))
print(rmse(y2,predictions))
print(mape(y2,predictions))
print(smape(y2,predictions))

0.09198155100162343
0.7719815176725909
0.5435671859569068


### Support Vector Regression

In [22]:
X_flattened = X_train.reshape((X_train.shape[0], -1))

In [23]:
svr_model = SVR(kernel='poly', degree=2, gamma=0.1,C=20,epsilon=0.01) 
svr_model.fit(X_flattened, y_train)

SVR(C=20, degree=2, epsilon=0.01, gamma=0.1, kernel='poly')

In [24]:
predictions = svr_model.predict(X2.reshape((X2.shape[0], -1)))
print(rmse(y2,predictions))
print(mape(y2,predictions))
print(smape(y2,predictions))

0.08760058657658516
0.390232696910985
0.3671115321489117


### ARIMA

In [25]:
arima_train_df = train_df[str(region)]
arima_test_df = test_df[str(region)]
arima_train_df_reshaped = arima_train_df.values.reshape(-1, 1)
arima_test_df_reshaped = arima_test_df.values.reshape(-1, 1)

In [26]:
# initialise scalers
scaler = MinMaxScaler(feature_range=(0.1,1))
scaler.fit(arima_train_df_reshaped)

arima_scaled_train = scaler.transform(arima_train_df_reshaped)
arima_scaled_test = scaler.transform(arima_test_df_reshaped)

In [27]:
p = 1
d = 1
q = 1

model = ARIMA(arima_scaled_train, order=(p, d, q))
model_fit = model.fit()

In [28]:
predictions = model_fit.predict(start=len(arima_scaled_train), end=len(arima_scaled_train) + len(arima_scaled_test) - 1, typ='levels')
print(rmse(arima_scaled_test.flatten(),predictions))
print(mape(arima_scaled_test.flatten(),predictions))
print(smape(arima_scaled_test.flatten(),predictions))

0.0671422379152212
0.2608799746141603
0.26342642700657853
